DEMO - ALERT DRIVER THROUGH EMAIL IF UNHEALTHY ENGINE CONDITION FOUND.

##### To be added :
Sender email

Receiver email

Password ( App password of sender's email )

In [ ]:
pip install imblearn

In [ ]:
pip install xgboost

In [ ]:
pip install catboost

In [ ]:
pip install shap

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix
from imblearn.combine import SMOTEENN
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss

file_path = '/content/engine_data.csv'
engine_data = pd.read_csv(file_path)

X = engine_data.drop('Engine Condition', axis=1)
y = engine_data['Engine Condition']

#Apply SMOTEENN to balance the entire dataset
smote_enn = SMOTEENN(random_state=42)
X_res_smoteenn, y_res_smoteenn = smote_enn.fit_resample(X, y)

X_train_res_smoteenn, X_test_res_smoteenn, y_train_res_smoteenn, y_test_res_smoteenn = train_test_split(X_res_smoteenn, y_res_smoteenn, test_size=0.2, random_state=42)

# Initialize models with the best hyperparameters
xgb_clf = XGBClassifier(
    alpha=0, colsample_bytree=0.7, lambda_=0, learning_rate=0.05,
    max_depth=7, min_child_weight=1, n_estimators=200, subsample=0.8,
    random_state=42
)

grad_boost_clf = GradientBoostingClassifier(
    learning_rate=0.1, max_depth=7, n_estimators=200, random_state=42
)

rf_clf = RandomForestClassifier(
    max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=200,
    random_state=42
)

log_reg_clf = LogisticRegression(
    C=1, penalty='l1', solver='liblinear', max_iter=100, random_state=42
)

# Base models
base_models = [
    ('xgboost', xgb_clf),
    ('gradient_boosting', grad_boost_clf),
    ('random_forest', rf_clf),
    ('logistic_regression', log_reg_clf)
]

# Meta-model (CatBoost with tuned parameters)
meta_model = CatBoostClassifier(
    iterations=100, learning_rate=0.05, depth=6, random_state=42, verbose=0
)

# Create a Stacking Classifier
stacked_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method='predict_proba',
    cv=5,
    n_jobs=-1
)

# Train the stacked model with validation set for early stopping
history = stacked_clf.fit(X_train_res_smoteenn, y_train_res_smoteenn)

# Make predictions on the test set
y_pred_test = stacked_clf.predict(X_test_res_smoteenn)
y_pred_test_prob = stacked_clf.predict_proba(X_test_res_smoteenn)[:, 1]

# Evaluation on Test Set
accuracy_test = accuracy_score(y_test_res_smoteenn, y_pred_test)
roc_auc_test = roc_auc_score(y_test_res_smoteenn, y_pred_test_prob)
conf_matrix_test = confusion_matrix(y_test_res_smoteenn, y_pred_test)
class_report_test = classification_report(y_test_res_smoteenn, y_pred_test)

print("\nTest Set - Stacked Ensemble Model - Evaluation Metrics:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"ROC-AUC: {roc_auc_test:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix_test)
print("\nClassification Report:")
print(class_report_test)



Test Set - Stacked Ensemble Model - Evaluation Metrics:
Accuracy: 0.8866
ROC-AUC: 0.9407

Confusion Matrix:
[[1040   87]
 [ 130  656]]

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      1127
           1       0.88      0.83      0.86       786

    accuracy                           0.89      1913
   macro avg       0.89      0.88      0.88      1913
weighted avg       0.89      0.89      0.89      1913



In [7]:
import joblib
joblib.dump(stacked_clf, 'engine_model.joblib')

['engine_model.joblib']

In [9]:
import numpy as np
import pandas as pd
import shap
import joblib  # To load the model
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Set the emergency threshold
threshold = 0.5

# Email credentials
sender_email = ""
receiver_email = ""
password = ""

# Function to send an emergency email
def send_emergency_email(critical_feature, critical_value, importance_score, health_condition_prob):
    subject = "🚨 Emergency Notification: Engine Condition Critical!"

    # Create a detailed body for the email
    body = f"""
    Dear Driver,

    🚨 Urgent: Your engine condition has been flagged as critical, and immediate attention is required.

    The predicted engine condition is Unhealthy, and the model has detected a potential failure based on the following:

    - Critical Parameter: {critical_feature}
    - Observed Value: {critical_value}
    - Feature Importance Score: {importance_score:.4f}
    - Probability of Healthy Condition: {health_condition_prob:.4f}

    The parameter listed above has the highest impact on the engine's current condition. We highly recommend pulling over and assessing the situation immediately to prevent any potential damage or further failure.

    Please seek assistance or contact support if needed.

    Stay Safe,
    The Engine Monitoring System
    """

    # Set up the MIME
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    try:
        # Connect to the Gmail SMTP server
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, password)  # Login to the email account
            server.sendmail(sender_email, receiver_email, message.as_string())
            print("🚨 Emergency email sent successfully!")

    except Exception as e:
        print(f"Error sending email: {e}")

# Set a random seed for reproducibility
np.random.seed(42)

# Generate random data for 20 samples
samples = {
    'Engine rpm': np.random.uniform(1000, 4000, 20),
    'Lub oil pressure': np.random.uniform(10, 80, 20),
    'Fuel pressure': np.random.uniform(30, 100, 20),
    'Coolant pressure': np.random.uniform(10, 40, 20),
    'lub oil temp': np.random.uniform(50, 120, 20),
    'Coolant temp': np.random.uniform(50, 120, 20),
}

# Create a DataFrame for the sample data
sample_data = pd.DataFrame(samples)

# Load the trained model
stacked_clf = joblib.load('engine_model.joblib')

# Predict using the loaded model
user_pred = stacked_clf.predict(sample_data)
user_pred_prob = stacked_clf.predict_proba(sample_data)[:, 1]  # probability of healthy condition

# Use SHAP to explain the model's prediction using KernelExplainer
explainer = shap.KernelExplainer(stacked_clf.predict_proba, sample_data)
shap_values = explainer.shap_values(sample_data)

# Display the prediction results and check for emergency
for i in range(len(sample_data)):
    condition = "Healthy" if user_pred[i] == 1 else "Unhealthy"
    print(f"\nPrediction Results for Sample {i+1}:")
    print(f"Predicted Engine Condition: {condition}")
    print(f"Predicted Probability of Healthy Condition: {user_pred_prob[i]:.4f}")

    # Check if an emergency notification is needed
    if user_pred_prob[i] < threshold:
        print("🚨 Emergency Notification: Engine condition is critical! Immediate attention required.")

        # Get SHAP values for the current sample
        shap_values_for_sample = shap_values[1][i]  # Assuming '1' corresponds to the unhealthy class

        # Find the most important feature
        critical_feature_index = np.argmax(np.abs(shap_values_for_sample))  # Feature with the highest absolute SHAP value
        critical_feature = sample_data.columns[critical_feature_index]
        critical_value = sample_data.iloc[i][critical_feature]
        importance_score = shap_values_for_sample[critical_feature_index]

        # Send the detailed emergency email
        send_emergency_email(critical_feature, critical_value, importance_score, user_pred_prob[i])

        # Stop processing further samples after detecting the critical condition
        break
    else:
        print("Engine is in healthy condition.")


  0%|          | 0/20 [00:00<?, ?it/s]


Prediction Results for Sample 1:
Predicted Engine Condition: Healthy
Predicted Probability of Healthy Condition: 0.7813
Engine is in healthy condition.

Prediction Results for Sample 2:
Predicted Engine Condition: Unhealthy
Predicted Probability of Healthy Condition: 0.1347
🚨 Emergency Notification: Engine condition is critical! Immediate attention required.
🚨 Emergency email sent successfully!
